## Selección de características para mejorar modelos predictivos

En este cuadernillo se realiza el tratamiento de datos del csv TITANIC, así como la implementación y comparación de varios algoritmos de búsqueda y entrenamiento.

### Tratamiento de datos(Hay que CAMBIAR)

Se aplicaron los siguientes procesos de preprocesamiento de datos:

    •Normalización de variables predictoras: Se normalizaron las variables Age y Fare utilizando el escalador MinMaxScaler para asegurar que todas las características estén en la misma escala.

    •Codificación numérica de atributos discretos: Los atributos Sex, Embarked, Alone y Deck, que originalmente se presentaban como cadenas de texto, fueron codificados numéricamente utilizando las técnicas de OrdinalEncoder o LabelEncoder, según corresponda.

### Primer experimento para evaluar la capacidad predictiva del conjunto de variables completo

    Para evaluar la calidad de las soluciones que se obtienen con los algoritmos de búsqueda de variables, se realizó un primer experimento en el que se entrenó un modelo de clasificación utilizando todas las variables predictoras disponibles. Se utilizó un árbol de decisión de clasificación.

### Algoritmos de búsqueda implementados

    •Búsqueda secuencial hacia atrás (backward_sequential_search): Este algoritmo busca encontrar el mejor subconjunto de variables predictoras eliminando iterativamente la variable que más afecta el rendimiento del modelo.

    •Búsqueda secuencial hacia atrás mixta (backward_sequential_search_mixto): Similar al anterior, pero también considera añadir variables si se mejora el rendimiento del modelo.

### Algoritmos de entrenamiento

    •Árboles de decisión de clasificación (DecisionTreeClassifier): Un algoritmo de aprendizaje supervisado utilizado para clasificación.




## Tratamiento de datos

In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler


model = RandomForestClassifier()

breast_cancer = pd.read_csv('../data/BreastCancer.csv')
breast_cancer.head()

atributos_continuos = ['mean radius' ,'mean texture' ,'mean perimeter' ,'mean area' ,'mean smoothness' ,'mean compactness' ,'mean concavity' ,
                       'mean concave points' ,'mean symmetry' ,'mean fractal dimension' ,'radius error' ,'texture error' ,'perimeter error' ,
                       'area error' ,'smoothness error' ,'compactness error' ,'concavity error' ,'concave points error','symmetry error','fractal dimension error',
                       'worst radius' ,'worst texture' ,'worst perimeter','worst area' ,'worst smoothness','worst compactness' ,'worst concavity' ,'worst concave points','worst symmetry','worst fractal dimension']
atributos = breast_cancer.loc[:, atributos_continuos]

objetivo = breast_cancer['diagnosis']
objetivo.head()  # objetivo es una Series unidimensional

normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)

# Como nos interesa conservar los atributos originales, realizamos la
# normalización sobre una copia del DataFrame de atributos
atributos_normalizados = atributos.copy()
atributos_normalizados[:] = normalizador.fit_transform(atributos_normalizados)
atributos_normalizados.head()

breast_cancer = atributos_normalizados.copy()
breast_cancer['Diagnosis'] = objetivo
breast_cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Diagnosis
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864,0
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878,0
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433,0
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711,0
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595,0


## Evaluación del modelo

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate


(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

### Árbol de decisión de clasificación(DecisionTreeClassifier)

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

clasificador_CART = DecisionTreeClassifier(random_state=42)
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'max_depth': range(3, 11),
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'min_samples_split': range(5, 20, 5)
}
clasificador_CART.fit(atributos_entrenamiento, objetivo_entrenamiento)

búsqueda_en_rejilla = GridSearchCV(clasificador_CART,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

# Obtener los mejores parámetros de la búsqueda en rejilla
mejores_parametros = búsqueda_en_rejilla.best_params_

# Guardar el valor de max_depth en una variable
max_depth_mejor = mejores_parametros.get('max_depth')
min_samples_split_mejor = mejores_parametros.get('min_samples_split')
mejor_score = búsqueda_en_rejilla.best_score_

print("Mejor valor de max_depth:", max_depth_mejor)
print("Mejor valor de min_samples_split:", min_samples_split_mejor)
print("Mejor score:", mejor_score)


Mejor valor de max_depth: 3
Mejor valor de min_samples_split: 15
Mejor score: 0.9277854245146335


In [20]:
resultados_validación_cruzada = cross_validate(clasificador_CART,
                                               atributos_entrenamiento,
                                               objetivo_entrenamiento,
                                               scoring='balanced_accuracy',
                                               cv=10, 
                                               n_jobs=-1,)
#resultados_validación_cruzada
resultados_validación_cruzada['test_score'].mean()

0.9229534917415242

In [21]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

# Supongamos que atributos es un DataFrame con las características y objetivo es una Serie con la variable objetivo
# Ajustar el modelo
clasificador_CART = DecisionTreeClassifier(max_depth=max_depth_mejor, min_samples_split=min_samples_split_mejor, random_state=42)
clasificador_CART.fit(atributos_entrenamiento, objetivo_entrenamiento)

# Calcular la precisión del modelo
score = clasificador_CART.score(atributos_entrenamiento, objetivo_entrenamiento)

# Obtener las predicciones
predicciones = clasificador_CART.predict(atributos_entrenamiento)

# Contar los valores de la variable objetivo
values = pd.Series(objetivo_entrenamiento).value_counts()

# Calcular la matriz de confusión
m_confusion = confusion_matrix(objetivo_entrenamiento, predicciones)

# Calcular la sensibilidad
predicciones_prueba = clasificador_CART.predict(atributos_prueba)
recallscore = recall_score(objetivo_prueba, predicciones_prueba)

# Imprimir los resultados
print(f'Variabes predictoras: \n{atributos_entrenamiento.columns}')
print()
print(f'Precisión: {score}')
print()
print(f'Valores: {values}')
print()
print(f'Matriz de confusión: \n{m_confusion}')
print()
print(f'Sensiilidad: {recallscore}')


Variabes predictoras: 
Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')

Precisión: 0.9582417582417583

Valores: diagnosis
1    285
0    170
Name: count, dtype: int64

Matriz de confusión: 
[[152  18]
 [  1 284]]

Sensiilidad: 0.9722222222222222


### KNN

In [22]:
from sklearn.compose import ColumnTransformer

normalizador = ColumnTransformer([('normalizador',
                                   MinMaxScaler(feature_range=(0, 1)),
                                   atributos_continuos)])

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

tubería_kNN = Pipeline([('preprocesador', normalizador),
                        ('kNN', KNeighborsClassifier())])
rejilla_de_parámetros = {
    # Número de vecinos impar (tarea de clasificación binaria)
    'kNN__n_neighbors': range(1, 10, 2),
    # Considerar las distancias Manhattan y euclídea
    'kNN__metric': ['manhattan', 'euclidean']
}


búsqueda_en_rejilla = GridSearchCV(tubería_kNN,
                                   rejilla_de_parámetros,
                                   scoring='balanced_accuracy',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

# Guardar el valor de max_depth en una variable
mejores_parametros = búsqueda_en_rejilla.best_params_

n_neighbors_mejor = mejores_parametros.get('kNN__n_neighbors')
metric_mejor = mejores_parametros.get('kNN__metric')
mejor_score = búsqueda_en_rejilla.best_score_

print("Mejor valor de n_neighbors:", n_neighbors_mejor)
print("Mejor valor de mmetric:", metric_mejor)
print("Mejor score:", mejor_score)



{'kNN__metric': 'euclidean', 'kNN__n_neighbors': 9}
Mejor valor de n_neighbors: 9
Mejor valor de mmetric: euclidean
Mejor score: 0.9688025210084034


In [33]:
clasificador_kNN = KNeighborsClassifier(n_neighbors=n_neighbors_mejor, metric=metric_mejor)

clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)


# Calcular la precisión del modelo
score = clasificador_kNN.score(atributos_entrenamiento, objetivo_entrenamiento)

# Obtener las predicciones
predicciones = clasificador_kNN.predict(atributos_entrenamiento)

# Contar los valores de la variable objetivo
values = pd.Series(objetivo_entrenamiento).value_counts()

# Calcular la matriz de confusión
m_confusion = confusion_matrix(objetivo_entrenamiento, predicciones)

# Calcular la sensibilidad
predicciones_prueba = clasificador_kNN.predict(atributos_prueba)
recallscore = recall_score(objetivo_prueba, predicciones_prueba)

print(f'Variabes predictoras: {atributos_entrenamiento.columns}')
print(f'Precisión: {score}')
print(f'Valores: {values}')
print(f'Matriz de confusión: \n{m_confusion}')
print()
print(f'Sensiilidad: {recallscore}')

Variabes predictoras: Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')
Precisión: 0.9318681318681319
Valores: diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[150  20]
 [ 11 274]]

Sensiilidad: 0.9861111111111112


## Algoritmo de búsqueda hacia atrás

In [ ]:
#Esto parametros nos permite tener el minimo num de variables de predictoras y un mayor score
import funciones.BusquedaSecuencialAtras as bsa
bsatras = bsa.backward_sequential_search(breast_cancer, 'Diagnosis', model, 1, 2)
bsatras


,variables,size,score
7,"[mean texture, mean concavity, perimeter error...",8,0.967042
14,"[mean texture, mean area, mean smoothness, mea...",15,0.967042
13,"[mean texture, mean area, mean smoothness, mea...",14,0.967042
8,"[mean texture, mean concavity, perimeter error...",9,0.965211
15,"[mean texture, mean area, mean smoothness, mea...",16,0.964683
12,"[mean texture, mean area, mean concavity, mean...",13,0.964683
10,"[mean texture, mean area, mean concavity, mean...",11,0.964241
16,"[mean texture, mean area, mean smoothness, mea...",17,0.964233
17,"[mean texture, mean area, mean smoothness, mea...",18,0.963279
11,"[mean texture, mean area, mean concavity, mean...",12,0.961874


### Árbol de decisión de clasificación(DecisionTreeClassifier)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

for k in range(0, len(bsatras)-1, 1):
    selected_variables = bsatras.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = breast_cancer.loc[:, selected_variables]
    
    # Dividir los datos
    X = breast_cancer[selected_variables]
    objetivo= breast_cancer['Diagnosis']
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    búsqueda_en_rejilla = GridSearchCV(clasificador_CART,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

    búsqueda_en_rejilla.fit(X_train, y_train)
    búsqueda_en_rejilla.fit(X_train, y_train)
    # Obtener los mejores parámetros de la búsqueda en rejilla
    mejores_parametros = búsqueda_en_rejilla.best_params_

    # Guardar el valor de max_depth en una variable
    max_depth_mejor = mejores_parametros.get('max_depth')
    min_samples_split_mejor = mejores_parametros.get('min_samples_split')
    mejor_score = búsqueda_en_rejilla.best_score_

    print("Mejor valor de max_depth:", max_depth_mejor)
    print("Mejor valor de min_samples_split:", min_samples_split_mejor)
    print("Mejor score:", mejor_score)



    # Entrenar el modelo
    clasificador_CART = DecisionTreeClassifier(
        max_depth=max_depth_mejor,  # Máxima profundidad del árbol
        min_samples_split=min_samples_split_mejor  # Mínimo número de ejemplos para poder particionar
    )

    resultados_validación_cruzada = cross_validate(clasificador_CART,
                                                X_train,
                                                y_train,
                                                scoring='balanced_accuracy',
                                                cv=10)

      # Ajustar el modelo
    clasificador_CART = DecisionTreeClassifier(max_depth=4)
    clasificador_CART.fit(X_train, y_train)


    # Calcular la precisión del modelo
    score = clasificador_CART.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_CART.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    # Calcular la sensibilidad
    predicciones_prueba = clasificador_CART.predict(X_test)
    recallscore = recall_score(y_test, predicciones_prueba)

    print(f'Variabes predictoras: \n{selected_variables}')
    print(f'Precisión: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()
    print()
    print(f'Sensiilidad: {recallscore}')

Variabes predictoras: 
['mean texture', 'mean concavity', 'perimeter error', 'symmetry error', 'worst texture', 'worst area', 'worst smoothness', 'worst symmetry']
Precisión: 0.9868131868131869
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[166   4]
 [  2 283]]


Sensiilidad: 1.0
Variabes predictoras: 
['mean texture', 'mean area', 'mean smoothness', 'mean concavity', 'mean symmetry', 'mean fractal dimension', 'perimeter error', 'smoothness error', 'concavity error', 'concave points error', 'symmetry error', 'worst texture', 'worst area', 'worst smoothness', 'worst symmetry']
Precisión: 0.9846153846153847
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[165   5]
 [  2 283]]


Sensiilidad: 0.9722222222222222
Variabes predictoras: 
['mean texture', 'mean area', 'mean smoothness', 'mean concavity', 'mean symmetry', 'mean fractal dimension', 'perimeter error', 'smoothness error', 'concavity error', 'symmetry erro

### KNN

In [ ]:
from sklearn.naive_bayes import CategoricalNB

clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)
for k in range(0, len(bsatras)-1, 1):

    selected_variables = bsatras.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = breast_cancer.loc[:, selected_variables]
    clasificador_kNN.fit(solucion, objetivo)
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)
 
    búsqueda_en_rejilla = GridSearchCV(clasificador_kNN,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

    búsqueda_en_rejilla.fit(X_train, y_train)
    
    búsqueda_en_rejilla.fit(X_train, y_train)
    # Obtener los mejores parámetros de la búsqueda en rejilla
    mejores_parametros = búsqueda_en_rejilla.best_params_

    # Guardar el valor de max_depth en una variable
    mejores_parametros = búsqueda_en_rejilla.best_params_

    n_neighbors_mejor = mejores_parametros.get('kNN__n_neighbors')
    metric_mejor = mejores_parametros.get('kNN__metric')
    mejor_score = búsqueda_en_rejilla.best_score_

    print("Mejor valor de n_neighbors:", n_neighbors_mejor)
    print("Mejor valor de mmetric:", metric_mejor)
    print("Mejor score:", mejor_score)

    clasificador_kNN = KNeighborsClassifier(
        # Para cada ejemplo se consideran los 5 ejemplos más cercanos
        n_neighbors=n_neighbors_mejor,
        # La cercanía viene determinada por la distancia euclídea
        metric=metric_mejor
    )

    # Entrenar el modelo
    clasificador_kNN.fit(X_train, y_train)

    # Calcular la precisión del modelo
    score = clasificador_kNN.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_kNN.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    #Calcular la sensibilidad
    predicciones_prueba = clasificador_kNN.predict(X_train)
    recallscore = recall_score(y_train, predicciones_prueba)

    print(f'Variabes predictoras: {selected_variables}')
    print(f'Precisión: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()
    print()
    print(f'Sensiilidad: {recallscore}')

Variabes predictoras: ['mean texture', 'mean concavity', 'perimeter error', 'symmetry error', 'worst texture', 'worst area', 'worst smoothness', 'worst symmetry']
Precisión: 0.9582417582417583
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[155  15]
 [  4 281]]


Sensiilidad: 0.9859649122807017
Variabes predictoras: ['mean texture', 'mean area', 'mean smoothness', 'mean concavity', 'mean symmetry', 'mean fractal dimension', 'perimeter error', 'smoothness error', 'concavity error', 'concave points error', 'symmetry error', 'worst texture', 'worst area', 'worst smoothness', 'worst symmetry']
Precisión: 0.9538461538461539
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[154  16]
 [  5 280]]


Sensiilidad: 0.9824561403508771
Variabes predictoras: ['mean texture', 'mean area', 'mean smoothness', 'mean concavity', 'mean symmetry', 'mean fractal dimension', 'perimeter error', 'smoothness error', 'concavity error', 's

## Algoritmo de búsqueda hacia atrás mixta

In [ ]:
import funciones.BusquedaSecuencialAtrasMixta as bsam
busq_atras_mixta = bsam.backward_sequential_mixed_search(breast_cancer, 'Diagnosis', model, 1, 2, 10)
busq_atras_mixta

KeyboardInterrupt: 

### Árbol de decisión de clasificación(DecisionTreeClassifier)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

for k in range(0, len(busq_atras_mixta)-1, 1):
    selected_variables = busq_atras_mixta.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = breast_cancer.loc[:, selected_variables]
    # Realizar la búsqueda secuencial hacia atrás
    model = DecisionTreeClassifier(max_depth=8)

    # Dividir los datos
    X = breast_cancer[selected_variables]
    objetivo= breast_cancer['Diagnosis']
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    búsqueda_en_rejilla = GridSearchCV(clasificador_kNN,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

    búsqueda_en_rejilla.fit(X_train, y_train)

    búsqueda_en_rejilla.fit(X_train, y_train)
    # Obtener los mejores parámetros de la búsqueda en rejilla
    mejores_parametros = búsqueda_en_rejilla.best_params_

    # Guardar el valor de max_depth en una variable
    max_depth_mejor = mejores_parametros.get('max_depth')
    min_samples_split_mejor = mejores_parametros.get('min_samples_split')
    mejor_score = búsqueda_en_rejilla.best_score_

    print("Mejor valor de max_depth:", max_depth_mejor)
    print("Mejor valor de min_samples_split:", min_samples_split_mejor)
    print("Mejor score:", mejor_score)


    # Entrenar el modelo
    clasificador_CART = DecisionTreeClassifier(
        max_depth=max_depth_mejor,  # Máxima profundidad del árbol
        min_samples_split=min_samples_split_mejor  # Mínimo número de ejemplos para poder particionar
    )

    resultados_validación_cruzada = cross_validate(clasificador_CART,
                                                X_train,
                                                y_train,
                                                scoring='balanced_accuracy',
                                                cv=10)
    
    # Entrenar el modelo
    clasificador_kNN.fit(X_train, y_train)

    # Ajustar el modelo
    clasificador_CART = DecisionTreeClassifier(max_depth=4)
    clasificador_CART.fit(X_train, y_train)

    # Calcular la precisión del modelo
    score = clasificador_CART.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_CART.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    #Calculamos la sensibilidad
    predicciones_prueba = clasificador_kNN.predict(X_train)
    recallscore = recall_score(y_train, predicciones_prueba)

    print(f'Variabes predictoras: \n{selected_variables}')
    print(f'Precisión: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()  
    print()
    print(f'Sensiilidad: {recallscore}')

Variabes predictoras: 
['mean symmetry', 'worst smoothness', 'radius error', 'mean perimeter', 'concave points error', 'worst symmetry', 'mean smoothness', 'area error', 'worst area', 'mean fractal dimension', 'worst texture', 'perimeter error', 'worst concavity', 'worst concave points', 'mean texture', 'worst fractal dimension', 'fractal dimension error']
Precisión: 0.9846153846153847
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[165   5]
 [  2 283]]


Sensiilidad: 0.9964912280701754
Variabes predictoras: 
['mean radius', 'mean texture', 'mean concavity', 'mean fractal dimension', 'area error', 'compactness error', 'symmetry error', 'fractal dimension error', 'worst texture', 'worst area', 'worst symmetry', 'worst fractal dimension', 'mean symmetry', 'worst smoothness']
Precisión: 0.9868131868131869
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[164   6]
 [  0 285]]


Sensiilidad: 0.9929824561403509
Varia

### KNN

In [ ]:
from sklearn.naive_bayes import CategoricalNB




clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)
for k in range(0, len(busq_atras_mixta)-1, 1):

    selected_variables = busq_atras_mixta.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = breast_cancer.loc[:, selected_variables]
    clasificador_kNN.fit(solucion, objetivo)
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    búsqueda_en_rejilla = GridSearchCV(clasificador_kNN,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

    búsqueda_en_rejilla.fit(X_train, y_train)

    búsqueda_en_rejilla.fit(X_train, y_train)
    # Obtener los mejores parámetros de la búsqueda en rejilla
    mejores_parametros = búsqueda_en_rejilla.best_params_

    # Guardar el valor de max_depth en una variable
    mejores_parametros = búsqueda_en_rejilla.best_params_

    n_neighbors_mejor = mejores_parametros.get('kNN__n_neighbors')
    metric_mejor = mejores_parametros.get('kNN__metric')
    mejor_score = búsqueda_en_rejilla.best_score_

    print("Mejor valor de n_neighbors:", n_neighbors_mejor)
    print("Mejor valor de mmetric:", metric_mejor)
    print("Mejor score:", mejor_score)

    clasificador_kNN = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 5 ejemplos más cercanos
    n_neighbors=n_neighbors_mejor,
    # La cercanía viene determinada por la distancia euclídea
    metric=metric_mejor
)


    # Entrenar el modelo
    clasificador_kNN.fit(X_train, y_train)

    # Calcular la precisión del modelo
    score = clasificador_kNN.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_kNN.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    # Calcular la sensibilidad
    predicciones_prueba = clasificador_kNN.predict(X_train)
    recallscore = recall_score(y_train, predicciones_prueba)

    print(f'Variabes predictoras: {selected_variables}')
    print(f'Precisión: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()
    print(f'Sensiilidad: {recallscore}')

Variabes predictoras: ['mean symmetry', 'worst smoothness', 'radius error', 'mean perimeter', 'concave points error', 'worst symmetry', 'mean smoothness', 'area error', 'worst area', 'mean fractal dimension', 'worst texture', 'perimeter error', 'worst concavity', 'worst concave points', 'mean texture', 'worst fractal dimension', 'fractal dimension error']
Precisión: 0.967032967032967
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[157  13]
 [  2 283]]

Sensiilidad: 0.9929824561403509
Variabes predictoras: ['mean radius', 'mean texture', 'mean concavity', 'mean fractal dimension', 'area error', 'compactness error', 'symmetry error', 'fractal dimension error', 'worst texture', 'worst area', 'worst symmetry', 'worst fractal dimension', 'mean symmetry', 'worst smoothness']
Precisión: 0.9714285714285714
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[158  12]
 [  1 284]]

Sensiilidad: 0.9964912280701754
Variabes p